In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import scipy.ndimage as ndimage
from scipy import interpolate
from matplotlib import cm
import copy
import random
import seaborn as sns
import glob
import cv2
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split


In [ ]:
class d3ERAUnetData(Dataset):
    def __init__(self, image_list, label_list, transformer):
        self.image_list = image_list
        self.label_list = label_list
        self.transformer = transformer
        
    def __getitem__(self, index):
        image = self.image_list[index]
        label = self.label_list[index]
        
        image_array = np.load(image)
        label_array = np.load(label)
        
        label_array[label_array > 0] = 1
        
        image_array = image_array.astype(np.float32)
        
        # Convert NumPy arrays to PyTorch tensors
        image_tensor = torch.from_numpy(image_array).unsqueeze(0)
        label_tensor = torch.from_numpy(label_array).long()  # Use long() for label
        
        image_tensor = self.transformer(image_tensor)
        label_tensor = self.transformer(label_tensor)
        
        return image_tensor, label_tensor
    
    def __len__(self):
        return len(self.image_list)


In [ ]:
class d3ERAUnet(nn.Module):
    def __init__(self, in_channels, n_classes, n_channels):
        super().__init__()
        self.in_channels = in_channels
        self.n_classes = n_classes
        self.n_channels = n_channels
        
        self.conv = nn.Sequential(
            DoubleConv(in_channels, n_channels),
            ResidualBlockEncoder(n_channels, n_channels)
        )
        self.enc1=Down(n_channels,2*n_channels)
        self.enc2=Down(2*n_channels,4*n_channels)
        self.enc3=Down(4*n_channels,8*n_channels)
        self.enc4=Down(8*n_channels,16*n_channels)
        
        self.bottleneck = DoubleConv(16*n_channels, 16*n_channels)
        
        self.dec1=Up(24*n_channels,8*n_channels)
        self.dec2=Up(12*n_channels,4*n_channels)
        self.dec3=Up(6*n_channels,2*n_channels)
        self.dec4=Up(3*n_channels,n_channels)
        
        self.out=Out(n_channels,n_classes)
        self.cbam = CBAM(16*n_channels)
        
    def forward(self,x):
        x1=self.conv(x)
        x2=self.enc1(x1)
        x3=self.enc2(x2)
        x4=self.enc3(x3)
        x5=self.enc4(x4)
        
        x_bottleneck = self.bottleneck(x5) 
        
        x_bottleneck = self.efam(x_bottleneck) 
        
        mask=self.dec1(x_bottleneck,x4)
        mask=self.dec2(mask,x3)
        mask=self.dec3(mask,x2)
        mask=self.dec4(mask,x1)
            
        mask=self.out(mask)
            
        return mask
    
    def get_attention_map(self, x):
        x1 = self.conv(x)
        x2 = self.enc1(x1)
        x3 = self.enc2(x2)
        x4 = self.enc3(x3)
        x5 = self.enc4(x4)
        
        x_bottleneck = self.bottleneck(x5)
        attention_map = self.cbam(x_bottleneck)
        
        return attention_map

In [ ]:
def dice_loss(pred, target, smooth=1e-6):
    pred = F.softmax(pred, dim=1)
    target = F.one_hot(target, num_classes=pred.shape[1]).permute(0, 4, 1, 2, 3).float()

    intersection = torch.sum(pred * target)
    total = torch.sum(pred) + torch.sum(target)
    
    dice = (2 * intersection + smooth) / (total + smooth)
    return 1 - dice

loss_fn=dice_loss

optimizer=torch.optim.Adam(model.parameters(),lr=0.0001)

epochs = 250